<a href="https://colab.research.google.com/github/riyaaaarane/adaptive_learning/blob/main/intern_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Data Generation

In [2]:
np.random.seed(42)
data = []
for _ in range(1000):
    current = random.choice([0, 1, 2])
    acc = np.clip(np.random.normal(0.7 - 0.1*current, 0.1), 0, 1)
    time_taken = np.clip(np.random.normal(10 + current*5, 3), 2, 25)
    prev_correct = 1 if random.random() < acc else 0

    # Generate next difficulty trend
    if acc > 0.8 and prev_correct:
        next_level = min(2, current + 1)
    elif acc < 0.5 or time_taken > 15:
        next_level = max(0, current - 1)
    else:
        next_level = current

    data.append([current, time_taken, acc, prev_correct, next_level])

df = pd.DataFrame(data, columns=["current_level", "avg_response_time", "recent_accuracy", "prev_correct", "next_level"])
print(df.head())

   current_level  avg_response_time  recent_accuracy  prev_correct  next_level
0              0           9.585207         0.749671             1           0
1              2          24.569090         0.564769             1           1
2              2          19.297589         0.476585             1           1
3              1          17.302304         0.757921             0           0
4              0          11.627680         0.653053             0           0


# Next Level Predictions

In [3]:
X = df[["current_level", "avg_response_time", "recent_accuracy", "prev_correct"]]
y = df["next_level"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000, multi_class='auto')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(f"Model trained with accuracy: {accuracy_score(y_test, y_pred):.2f}")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model trained with accuracy: 0.85


# Puzzle Generator

In [4]:
def generate_puzzle(level):
    if level == 0:
        a, b = random.randint(1, 10), random.randint(1, 10)
        question = f"{a} + {b}"
        answer = a + b
    elif level == 1:
        a, b = random.randint(5, 15), random.randint(1, 10)
        question = f"{a} - {b}"
        answer = a - b
    else:
        a, b = random.randint(2, 10), random.randint(2, 10)
        question = f"{a} × {b}"
        answer = a * b
    return question, answer

#Adaptive Learning Session

In [11]:
print("\n Welcome to Math Adventures — AI Adaptive Mode!")
name = input("Enter your name: ")
current_level = int(input("Choose starting difficulty (0=Easy, 1=Medium, 2=Hard): "))

session_log = []
n_questions = 5

for i in range(n_questions):
    print(f"\nQuestion {i+1} | Difficulty: {['Easy','Medium','Hard'][current_level]}")
    q, ans = generate_puzzle(current_level)
    print(f"Solve: {q}")

    start = time.time()
    try:
        user_ans = int(input("Your answer: "))
    except ValueError:
        user_ans = None
    end = time.time()

    correct = 1 if user_ans == ans else 0
    response_time = round(end - start, 2)
    print("Correct!" if correct else f"Wrong. Correct answer: {ans}")

    # Calculate rolling metrics
    recent_accuracy = np.mean([x["correct"] for x in session_log[-3:]] or [1])

    # ML Model predicts next difficulty (pure ML — no rules)
    X_live = np.array([[current_level, response_time, recent_accuracy, correct]])
    next_level = int(model.predict(X_live)[0])

    session_log.append({
        "question": q,
        "answer": ans,
        "user_answer": user_ans,
        "correct": correct,
        "time": response_time,
        "current_level": current_level,
        "next_level": next_level
    })

    current_level = next_level  # model decides


 Welcome to Math Adventures — AI Adaptive Mode!
Enter your name: riya
Choose starting difficulty (0=Easy, 1=Medium, 2=Hard): 1

Question 1 | Difficulty: Medium
Solve: 13 - 2
Your answer: 5


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Wrong. Correct answer: 11

Question 2 | Difficulty: Medium
Solve: 15 - 9
Your answer: 6


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Correct!

Question 3 | Difficulty: Medium
Solve: 9 - 7
Your answer: 2


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Correct!

Question 4 | Difficulty: Medium
Solve: 5 - 5
Your answer: 0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


Correct!

Question 5 | Difficulty: Medium
Solve: 7 - 7
Your answer: 1
Wrong. Correct answer: 0


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


# Session Summary

In [12]:
df_sess = pd.DataFrame(session_log)
accuracy = df_sess["correct"].mean()
avg_time = df_sess["time"].mean()

print("\nSession Summary")
print(df_sess[["question", "user_answer", "correct", "time", "current_level", "next_level"]])
print(f"\nOverall Accuracy: {accuracy*100:.1f}%")
print(f"Average Response Time: {avg_time:.2f}s")

# Recommend next difficulty
recommended = int(df_sess["next_level"].iloc[-1])
print(f"Next Recommended Difficulty: {['Easy','Medium','Hard'][recommended]}")


Session Summary
  question  user_answer  correct  time  current_level  next_level
0   13 - 2            5        0  2.54              1           1
1   15 - 9            6        1  6.94              1           1
2    9 - 7            2        1  2.32              1           1
3    5 - 5            0        1  3.85              1           1
4    7 - 7            1        0  1.77              1           1

Overall Accuracy: 60.0%
Average Response Time: 3.48s
Next Recommended Difficulty: Medium
